# plumbum Tutorial

This notebook walks through the core ideas behind **plumbum** pipelines.
It covers synchronous operators, iterable helpers, and the async variants.


## Getting Started

Operators are regular Python callables decorated with `@pb`. Data is threaded
into the first argument using the `>>` operator.


In [ ]:
from pdum.plumbum import pb
from pdum.plumbum.iterops import tee


@pb
def add(value: int, amount: int) -> int:
    return value + amount


@pb
def multiply(value: int, factor: int) -> int:
    return value * factor

### Building Pipelines


In [ ]:
pipeline = add(3) | multiply(2)
5 >> pipeline

### Partial Application

Operators can be partially applied. The accumulated arguments are appended
until you thread a value through the pipeline.


In [ ]:
increment_then_double = add(1) | multiply(2)
list(value >> increment_then_double for value in [1, 2, 3])

## Iterable Helpers

`pdum.plumbum.iterops` exposes ready-made operators for working with
iterables. They compose like normal operators.


In [ ]:
from pdum.plumbum.iterops import select, where, batched

numbers = [1, 2, 3, 4, 5]
pipeline = select(lambda value: value * 2) | where(lambda value: value % 3 != 0)
numbers >> (pipeline | pb(list))

In [ ]:
numbers >> (batched(2) | pb(list))

## Inspecting Pipelines

Use `tee` to observe intermediate values without breaking the flow.


In [ ]:
numbers >> (select(lambda value: value + 1) | tee | where(lambda value: value % 2 == 0) | pb(list))

## Async Pipelines

Async operators use the `@apb` decorator. You can await the threaded
result directly from a notebook cell.


In [ ]:
from pdum.plumbum import apb


@apb
async def async_double(value: int) -> int:
    return value * 2


await (5 >> (async_double | add(3)))

## Async Iterable Helpers

The `pdum.plumbum.aiterops` module mirrors the iterable helpers with async
            variants prefixed by `a`.


In [ ]:
from pdum.plumbum.aiterops import aiter, aselect, awhere


async def collect_async_values():
    pipeline = aiter | aselect(lambda value: value + 1) | awhere(lambda value: value % 2 == 0)
    iterator = await ([1, 2, 3, 4] >> pipeline)
    return [item async for item in iterator]


await collect_async_values()

Async helpers accept synchronous or asynchronous callables just like the
synchronous API.
